In [74]:
import os
import sys

import oras.defaults
import oras.oci
import oras.provider
import oras.container
from oras.decorator import ensure_container
import oras.defaults
import json
import logging
from dataclasses import asdict
from pprint import pprint
import requests
import hashlib

logger = logging.getLogger(__name__)
logging.basicConfig(level=20)

def get_string_hash(input_string: str, algorithm: str = "sha256") -> str:
    """
    Return a hash of the input string based on the specified algorithm.
    Raises AttributeError if an incorrect algorithm is supplied.

    :param input_string: the string to hash
    :type input_string: str
    :param algorithm: the algorithm to use
    :type algorithm: str
    """
    hasher = getattr(hashlib, algorithm)()
    hasher.update(input_string.encode('utf-8'))
    return hasher.hexdigest()

def get_oras_client():
    """
    Consistent method to get an oras client
    """
    user = os.environ.get("ORAS_USER")
    password = os.environ.get("ORAS_PASS")
    reg = Registry()
    if user and password:
        print("Found username and password for basic auth")
        reg.set_basic_auth(user, password)
    else:
        logger.warning("No ORAS_USER or ORAS_PASS defined, no auth.")
    return reg


class Registry(oras.provider.Registry):
    def __init__(self):
        oras.provider.Registry.__init__(self, insecure=True)

    @ensure_container
    def push(self, container, archives: list, m_annotations: dict, **kwargs):
        """
        Given a list of layer metadata (paths and corresponding mediaType) push.
        """
        # Prepare a new manifest
        manifest = oras.oci.NewManifest()
        

        # Upload files as blobs
        for item in archives:
            logger.info(f"parsing {item}.")
            blob = item.get("path")
            media_type = (
                item.get("media_type") or oras.defaults.default_blob_media_type
            )
            annots = item.get("annotations") or {}

            if not blob or not os.path.exists(blob):
                logger.warning(f"Path {blob} does not exist or is not defined.")
                continue

            # Artifact title is basename or user defined
            blob_name = item.get("title") or os.path.basename(blob)

            # If it's a directory, we need to compress
            cleanup_blob = False
            if os.path.isdir(blob):
                blob = oras.utils.make_targz(blob)
                cleanup_blob = True

            # Create a new layer from the blob
            layer = oras.oci.NewLayer(blob, media_type, is_dir=cleanup_blob)
            logger.debug(f"Preparing layer {layer}")

            # Update annotations with title we will need for extraction
            annots.update({oras.defaults.annotation_title: blob_name})
            layer["annotations"] = annots

            # update the manifest with the new layer
            manifest["layers"].append(layer)

            # Upload the blob layer
            logger.info(f"Uploading {blob} to {container.uri}")
            response = self.upload_blob(blob, container, layer)
            self._check_200_response(response)

            # Do we need to cleanup a temporary targz?
            if cleanup_blob and os.path.exists(blob):
                os.remove(blob)

        # Prepare manifest and config (add your custom annotations here)
        manifest["annotations"] = m_annotations or {}

        subject = kwargs.get("subject")
        if subject:
            manifest["subject"] = asdict(subject)

        conf, config_file = oras.oci.ManifestConfig()
        conf["annotations"] = {}

        # Config is just another layer blob!
        logger.debug(f"Preparing config {conf}")
        with (
            oras.provider.temporary_empty_config()
            if config_file is None
            else oras.provider.nullcontext(config_file)
        ) as config_file:
            print(config_file)
            response = self.upload_blob(
                config_file, container, conf
            )
            self._check_200_response(response)

        # Final upload of the manifest
        manifest["config"] = conf

        print("Manifest:")
        # pprint(manifest)
        manifest_as_str = json.dumps(manifest, ensure_ascii=False, separators=(', ', ': '))
        print(manifest_as_str)
        expected_hash = get_string_hash(manifest_as_str)
        print("expected hash: "+expected_hash)
        if subject:
            logger.warn(f"REWRITING container from {container}")
            container = oras.container.Container(container.uri + "@sha256:" + expected_hash)
        logger.info(f"container is: {container}")

        self._check_200_response(self.upload_manifest(manifest, container))
        print(f"Successfully pushed {container}")
        return response

In [75]:
from datetime import datetime
import oras.utils

def push(uri, root, m_annotations, **kwargs):
    """
    Given an ORAS identifier, save artifacts to it.
    """
    oras_cli = get_oras_client()

    # Create lookup of archives - relative path and mediatype
    archives = []
    now = datetime.now()

    # Using os.listdir assumes we have single files at the base of our root.
    for filename in os.listdir(root):
        logger.info(f"parsing {filename}.")
        # use some logic here to derive the mediaType
        media_type = "application/vnd.org.dinosaur.tool.datatype.v1+json"

        # Add some custom annotations!
        size = os.path.getsize(os.path.join(root, filename))  # bytes
        annotations = {"creationTime": str(now), "size": str(size)}
        archives.append(
            {
                "path": filename,
                "title": filename,
                "media_type": media_type,
                "annotations": annotations,
             }
         )

    # Push should be relative to cache context
    with oras.utils.workdir(root):
        oras_cli.push(uri, archives, m_annotations, **kwargs)

In [76]:
m_annotations = {}
with open("annotations.json", "r") as file:
    m_annotations = json.load(file)['$manifest']

push("localhost:8080/dinosaur/artifact:v1", os.getcwd()+"/models", m_annotations)

INFO:__main__:parsing mymodel.onnx.
INFO:__main__:parsing {'path': 'mymodel.onnx', 'title': 'mymodel.onnx', 'media_type': 'application/vnd.org.dinosaur.tool.datatype.v1+json', 'annotations': {'creationTime': '2024-05-17 22:23:52.647275', 'size': '421403'}}.
INFO:__main__:Uploading mymodel.onnx to localhost:8080/dinosaur/artifact:v1
INFO:__main__:container is: localhost:8080/dinosaur/artifact:v1


/var/folders/n0/3c71vqs570b8l21fmnvb5k640000gn/T/tmp7m0ins99/oras-tmp.8q3odx1g/_khyml7u.json
Manifest:
{"schemaVersion": 2, "mediaType": "application/vnd.oci.image.manifest.v1+json", "config": {"mediaType": "application/vnd.unknown.config.v1+json", "size": 2, "digest": "sha256:44136fa355b3678a1146ad16f7e8649e94fb4fc21fe77e8310c060f61caaff8a", "annotations": {}}, "layers": [{"mediaType": "application/vnd.org.dinosaur.tool.datatype.v1+json", "size": 421403, "digest": "sha256:2536d32a8d81c95b8261926e57e34e00b32a7ea22eef96c653ce29be1e64f7e1", "annotations": {"creationTime": "2024-05-17 22:23:52.647275", "size": "421403", "org.opencontainers.image.title": "mymodel.onnx"}}], "annotations": {"mr_str:id": "0", "mr_str:name": "Investment Portfolio Optimization Model", "mr_str:description": "This is a first installment of the Investment portfolio opt. model\n", "mr_str:owner": "mmortari", "mr_json:customProperties": "{\"my-prop1\": {\"metadataType\": \"MetadataStringValue\", \"string_value\": \"

In [77]:
# using HEAD
response = requests.head("http://localhost:8080/v2/dinosaur/artifact/manifests/v1")
print("Response Headers:")
for header, value in response.headers.items():
    print(f"{header}: {value}")
print(response)

Response Headers:
Access-Control-Allow-Headers: Authorization,content-type,X-ZOT-API-CLIENT
Access-Control-Allow-Methods: HEAD,GET,DELETE,OPTIONS
Access-Control-Allow-Origin: *
Content-Length: 1467
Content-Type: application/vnd.oci.image.manifest.v1+json
Docker-Content-Digest: sha256:a80088f232fe06f2705eba1b16badbf09b041fdf4065d71d54e09bad0485b261
Date: Fri, 17 May 2024 20:23:52 GMT
<Response [200]>


In [78]:
# using GET
response = requests.get("http://localhost:8080/v2/dinosaur/artifact/manifests/v1")
print("Response Headers:")
for header, value in response.headers.items():
    print(f"{header}: {value}")
print(json.dumps(json.loads(response.text), indent=2))
digest = response.headers['Docker-Content-Digest']
content_length = response.headers['Content-Length']

Response Headers:
Access-Control-Allow-Origin: *
Content-Length: 1467
Content-Type: application/vnd.oci.image.manifest.v1+json
Docker-Content-Digest: sha256:a80088f232fe06f2705eba1b16badbf09b041fdf4065d71d54e09bad0485b261
Date: Fri, 17 May 2024 20:23:52 GMT
{
  "schemaVersion": 2,
  "mediaType": "application/vnd.oci.image.manifest.v1+json",
  "config": {
    "mediaType": "application/vnd.unknown.config.v1+json",
    "size": 2,
    "digest": "sha256:44136fa355b3678a1146ad16f7e8649e94fb4fc21fe77e8310c060f61caaff8a",
    "annotations": {}
  },
  "layers": [
    {
      "mediaType": "application/vnd.org.dinosaur.tool.datatype.v1+json",
      "size": 421403,
      "digest": "sha256:2536d32a8d81c95b8261926e57e34e00b32a7ea22eef96c653ce29be1e64f7e1",
      "annotations": {
        "creationTime": "2024-05-17 22:23:52.647275",
        "size": "421403",
        "org.opencontainers.image.title": "mymodel.onnx"
      }
    }
  ],
  "annotations": {
    "mr_str:id": "0",
    "mr_str:name": "Investm

In [79]:
oras_cli = get_oras_client()
m = oras_cli.get_manifest("localhost:8080/dinosaur/artifact@" + digest)
pprint(m)

{'annotations': {'mr_json:customProperties': '{"my-prop1": {"metadataType": '
                                             '"MetadataStringValue", '
                                             '"string_value": "ciao"}, '
                                             '"my-prop2": {"metadataType": '
                                             '"MetadataDoubleValue", '
                                             '"double_value": 99.999}, '
                                             '"Portfolio Optimization": '
                                             '{"metadataType": '
                                             '"MetadataStringValue", '
                                             '"string_value": ""}, "Investment '
                                             'Strategy": {"metadataType": '
                                             '"MetadataStringValue", '
                                             '"string_value": ""}, '
                                             '"Fin

In [80]:
# by THIS notebook convention, will be used as 
# attach
push("localhost:8080/dinosaur/artifact", os.getcwd()+"/annotations", None, subject=oras.provider.Subject(mediaType="application/vnd.oci.image.manifest.v1+json", digest=digest, size=int(content_length)))

INFO:__main__:parsing metadata.yml.
INFO:__main__:parsing {'path': 'metadata.yml', 'title': 'metadata.yml', 'media_type': 'application/vnd.org.dinosaur.tool.datatype.v1+json', 'annotations': {'creationTime': '2024-05-17 22:23:52.747140', 'size': '24'}}.
INFO:__main__:Uploading metadata.yml to localhost:8080/dinosaur/artifact:latest
/var/folders/n0/3c71vqs570b8l21fmnvb5k640000gn/T/ipykernel_41137/4099588270.py:137: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  logger.warn(f"REWRITING container from {container}")
INFO:__main__:container is: localhost:8080/dinosaur/artifact@sha256:125b95566f89eebd2cd2f4d010d9bba019d0c94446a3eabfddd7bfcd9045c6d3


/var/folders/n0/3c71vqs570b8l21fmnvb5k640000gn/T/tmp8xah9qyh/oras-tmp.n9n_m7ls/xyfylueu.json
Manifest:
{"schemaVersion": 2, "mediaType": "application/vnd.oci.image.manifest.v1+json", "config": {"mediaType": "application/vnd.unknown.config.v1+json", "size": 2, "digest": "sha256:44136fa355b3678a1146ad16f7e8649e94fb4fc21fe77e8310c060f61caaff8a", "annotations": {}}, "layers": [{"mediaType": "application/vnd.org.dinosaur.tool.datatype.v1+json", "size": 24, "digest": "sha256:1b1fc16e9c2d28eba8c4207373d0335f25147e95aa1f03257574e4c990be490b", "annotations": {"creationTime": "2024-05-17 22:23:52.747140", "size": "24", "org.opencontainers.image.title": "metadata.yml"}}], "annotations": {}, "subject": {"mediaType": "application/vnd.oci.image.manifest.v1+json", "digest": "sha256:a80088f232fe06f2705eba1b16badbf09b041fdf4065d71d54e09bad0485b261", "size": 1467}}
expected hash: 125b95566f89eebd2cd2f4d010d9bba019d0c94446a3eabfddd7bfcd9045c6d3
Successfully pushed localhost:8080/dinosaur/artifact@sha256:

In [81]:
# now showcase the ATTACH-ed to the dinosaur/artifact:v1
response = requests.get("http://localhost:8080/v2/dinosaur/artifact/referrers/"+digest)
print("Response Headers:")
for header, value in response.headers.items():
    print(f"{header}: {value}")
print(json.dumps(json.loads(response.text), indent=2))

Response Headers:
Access-Control-Allow-Headers: Authorization,content-type,X-ZOT-API-CLIENT
Access-Control-Allow-Methods: GET,OPTIONS
Access-Control-Allow-Origin: *
Content-Type: application/vnd.oci.image.index.v1+json
Date: Fri, 17 May 2024 20:23:52 GMT
Content-Length: 296
{
  "schemaVersion": 2,
  "mediaType": "application/vnd.oci.image.index.v1+json",
  "manifests": [
    {
      "mediaType": "application/vnd.oci.image.manifest.v1+json",
      "digest": "sha256:125b95566f89eebd2cd2f4d010d9bba019d0c94446a3eabfddd7bfcd9045c6d3",
      "size": 756,
      "artifactType": "application/vnd.unknown.config.v1+json"
    }
  ]
}


GraphQL
```gql
{
  Image(image: "dinosaur/artifact:v1") {
    RepoName
    Tag
    Digest
    MediaType
    Size
    DownloadCount
    LastUpdated
    Description
    IsSigned
    Licenses
    Labels
    Title
    Source
    Documentation
    Vendor
    Authors
    IsDeletable
    Manifests {
      Referrers {
        Digest
      }
      Digest
      ConfigDigest
      LastUpdated
      Size
      IsSigned
      DownloadCount
      ArtifactType
      Layers {
        Size
        Digest
      }
    }
    Referrers {
      Annotations {
        Key
        Value
      }
      MediaType
      ArtifactType
      Size
      Digest
    }
  }
}
```

response:
```json
{
  "data": {
    "Image": {
      "RepoName": "dinosaur/artifact",
      "Tag": "v1",
      "Digest": "sha256:a80088f232fe06f2705eba1b16badbf09b041fdf4065d71d54e09bad0485b261",
      "MediaType": "application/vnd.oci.image.manifest.v1+json",
      "Size": "422872",
      "DownloadCount": 2,
      "LastUpdated": null,
      "Description": "",
      "IsSigned": false,
      "Licenses": "",
      "Labels": "",
      "Title": "",
      "Source": "",
      "Documentation": "",
      "Vendor": "",
      "Authors": "",
      "IsDeletable": null,
      "Manifests": [
        {
          "Referrers": [
            {
              "Digest": "sha256:125b95566f89eebd2cd2f4d010d9bba019d0c94446a3eabfddd7bfcd9045c6d3"
            }
          ],
          "Digest": "sha256:a80088f232fe06f2705eba1b16badbf09b041fdf4065d71d54e09bad0485b261",
          "ConfigDigest": "sha256:44136fa355b3678a1146ad16f7e8649e94fb4fc21fe77e8310c060f61caaff8a",
          "LastUpdated": null,
          "Size": "422872",
          "IsSigned": false,
          "DownloadCount": 2,
          "ArtifactType": "application/vnd.unknown.config.v1+json",
          "Layers": [
            {
              "Size": "421403",
              "Digest": "sha256:2536d32a8d81c95b8261926e57e34e00b32a7ea22eef96c653ce29be1e64f7e1"
            }
          ]
        }
      ],
      "Referrers": [
        {
          "Annotations": [],
          "MediaType": "application/vnd.oci.image.manifest.v1+json",
          "ArtifactType": "application/vnd.unknown.config.v1+json",
          "Size": 756,
          "Digest": "sha256:125b95566f89eebd2cd2f4d010d9bba019d0c94446a3eabfddd7bfcd9045c6d3"
        }
      ]
    }
  }
}
```

![image.png](./Screenshot%202024-05-17%20at%2022.33.18.png)
![image.png](./Screenshot%202024-05-17%20at%2022.33.28.png)
![image.png](./Screenshot%202024-05-17%20at%2022.33.36.png)